# Нахождение собственных значений (метод степенной итерации с дефляцией)
---
Собственные значения матрицы $C$ можно найти не только через решение уравнения $\det(C - \lambda I) = 0$, но и с помощью итеративных численных методов. В частности, **метод степенной итерации** (power iteration) позволяет найти наибольшее по модулю собственное значение и соответствующий ему собственный вектор. А затем, применяя **дефляцию**, можно последовательно «откалывать» уже найденные собственные пары и получать оставшиеся значения.

### Идея метода степенной итерации

1. **Инициализация**
   Задаём случайный вектор $b^{(0)} \in \mathbb{R}^n$ (не равный нулю) и нормируем его:
   $$
     b^{(0)} \leftarrow \frac{b^{(0)}}{\|b^{(0)}\|}.
   $$

2. **Итерации**
   На каждом шаге вычисляем
   $$
     y = A\,b^{(k)},\quad
     b^{(k+1)} = \frac{y}{\|y\|}.
   $$
   Обновлённый вектор $b^{(k+1)}$ стремится к собственному вектору, соответствующему наибольшему по модулю собственному значению.

3. **Оценка собственного значения**
   Используем **кратное отношение Релея** (Rayleigh quotient):
   $$
     \lambda^{(k+1)}
       = \frac{(b^{(k+1)})^\top A\, b^{(k+1)}}{(b^{(k+1)})^\top b^{(k+1)}}
       = (b^{(k+1)})^\top A\, b^{(k+1)},
   $$
   и проверяем условие сходимости
   $$
     \bigl|\lambda^{(k+1)} - \lambda^{(k)}\bigr| < \text{tol}.
   $$

### Дефляция

После того как найдено собственное значение $\lambda$ и соответствующий ему нормированный вектор $b$, модифицируем матрицу:
$$
  A \leftarrow A - \lambda\,b\,b^\top.
$$
Это «вычёркивает» найденную компоненту из спектра матрицы. Повторяя процедуру на обновлённой $A$, мы последовательно извлекаем все $n$ собственных значений.

---

### Особенности реализации

- **Сходимость** метода степенной итерации гарантируется для матриц, у которых одно собственное значение строго превосходит все остальные по модулю.
- **Дефляция** сохраняет симметричность $A$, если исходная матрица симметрична, и корректно устраняет найденную компоненту.
- **Параметр `tol`** определяет требуемую точность для собственного значения.
- **Ограничение числа итераций** (1000) предотвращает бесконечные циклы на плохо обусловленных матрицах.
- При кратных собственных значениях алгоритм может несколько раз сходиться к одному и тому же значению с небольшой разницей — при необходимости стоит объединять близкие оценки.

---

In [2]:
from src.Matrix import Matrix
from typing import List
import math
import random

In [3]:
def find_eigenvalues(C: 'Matrix', tol: float = 1e-6) -> List[float]:
    """
    Находит все собственные значения матрицы C методом power iteration с дефляцией.
    Возвращает список собственных значений, упорядоченных по убыванию.
    """
    n = C.rows
    A = [[C.data[i][j] for j in range(n)] for i in range(n)]
    eigenvalues: List[float] = []
    for _ in range(n):
        b = [random.random() for _ in range(n)]
        norm_b = math.sqrt(sum(x*x for x in b)) or 1.0
        b = [x / norm_b for x in b]
        lambda_old = 0.0
        for _ in range(1000):
            Ab = [sum(A[i][j] * b[j] for j in range(n)) for i in range(n)]
            norm_ab = math.sqrt(sum(x*x for x in Ab)) or 1.0
            b = [x / norm_ab for x in Ab]
            # Rayleigh quotient
            lambda_new = sum(b[i] * sum(A[i][j] * b[j] for j in range(n)) for i in range(n))
            if abs(lambda_new - lambda_old) < tol:
                break
            lambda_old = lambda_new
        eigenvalues.append(lambda_new)
        for i in range(n):
            for j in range(n):
                A[i][j] -= lambda_new * b[i] * b[j]
    return eigenvalues


---